# CCD

In [ ]:
def ccd(model, dist):
    samples = [[] for param in model.trainable_parameters]
    param_vals = []
    multipliers = np.array([1/dist, 1, dist])
    for param in model.trainable_parameters:
        param_value = param.numpy()
        new_vals = param_value * multipliers
        new_vals = np.transpose(new_vals)
        param_vals.append(list(new_vals))
    for combination in itertools.product(*param_vals):
        for i, param in enumerate(combination):
            samples[i].append(param)
    for i in range(len(samples)):
        samples[i] = np.vstack(samples[i])
    return samples

In [ ]:
parameter_samples_2 = ccd(model, 4)
num_samples = parameter_samples_2[-1].shape[0]
likelihoods = []
for i in range(num_samples):
    # Note that hmc_helper.current_state contains the unconstrained variables
    for var, var_samples in zip(model.trainable_parameters, parameter_samples_2):
        var.assign(var_samples[i].reshape(-1, 1))
    likelihood = model.log_posterior_density()
    likelihoods.append(likelihood)
for i in range(num_samples):
    for var, var_samples in zip(model.trainable_parameters, parameter_samples_2):
        var.assign(var_samples[i])
    weight = model.log_posterior_density()
    f_batch = model.predict_f_samples(X_test, 5)
    f_samples.append(f_batch)